# Customer Support Goal-Oriented Multi Agent Flow (CrewAI)

### **Problem Statement**
Customer support teams often receive diverse queries—from login issues and billing errors to technical problems. Manually analyzing each message, searching a knowledge base, generating a helpful response, and deciding whether to escalate takes time and is inconsistent across agents.

### **Approach**
This workflow uses a **goal-oriented multi-agent system** to automate support resolution. The flow dynamically analyzes the user's query using four cooperating AI agents:

1. **Issue Identification Agent**  
   Classifies the query by intent, urgency, and sentiment.

2. **Knowledge Search Agent**  
   Searches relevant internal documentation or solutions.

3. **Solution Generation Agent**  
   Drafts a clear, user-friendly support response.

4. **Escalation Agent**  
   Evaluates if the AI response is sufficient or requires human review.

User input (`customer_query`) drives the reasoning across agents, making the workflow adaptable to real-world support scenarios.

✔ Faster support  
✔ Personalized responses  
✔ Smart escalation logic


For running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [52]:
from crewai import Agent, Task, Crew

In [50]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: issue_identifier
### Agent: knowledgebase_searcher
### Agent: solution_generator
### Agent: escalation_agent

In [53]:

issue_classifier = Agent(
    role="Issue Identification Specialist",
    goal="Correctly interpret and classify customer issues.",
    backstory="You are trained in identifying user intent and support category.",
    llm = llm,
    verbose=True
)

knowledge_agent = Agent(
    role="Knowledge Base Specialist",
    goal="Search the internal documentation to find matching solutions.",
    backstory="You are a search system that returns the best match from the support knowledge base.",
    llm=llm,
    verbose=True
)

solution_agent = Agent(
    role="Solution Generator",
    goal="Craft a final, user-friendly response using available info.",
    backstory="You write empathetic support replies with clarity and helpful tone.",
    llm=llm,
    verbose=True
)

escalation_agent = Agent(
    role="Escalation Coordinator",
    goal="Determine whether escalation to a human agent is required.",
    backstory="You evaluate uncertainty, urgency, or missing details to decide escalation.",
    llm=llm,
    verbose=True
)


In [54]:
# ---------------- Tasks ----------------
identify_issue = Task(
    description="""Analyze the customer message: "{customer_query}".
Identify:
- Issue type (billing, technical, login, refund, general inquiry)
- Sentiment (angry, neutral, confused, urgent)
- Priority level (low, normal, high)
Output as JSON with keys: type, urgency, sentiment, summary.""",
    agent=issue_classifier,
    expected_output="A JSON object describing classified issue details."
)

fetch_knowledge = Task(
    description="""Based on the classification above and the message "{customer_query}",
search the knowledge base and retrieve the closest matching solution.
If no solution exists, respond with: 'NO MATCH FOUND'.""",
    agent=knowledge_agent,
    expected_output="Best possible KB solution or 'NO MATCH FOUND'."
)

generate_solution_response = Task(
    description="""Using the classification and retrieved KB content for: "{customer_query}",
create a friendly step-by-step customer support response that:
- Uses plain language
- Offers clear next steps
- Includes apology if needed""",
    agent=solution_agent,
    expected_output="A polished support response formatted for users."
)

apply_escalation_logic = Task(
    description="""Evaluate whether the AI response is sufficient.
Consider:
- Confidence level
- Urgency from issue classification
- Whether solution was 'NO MATCH FOUND'

Output either:
- `resolved` if handled successfully
- `requires_escalation` with explanation and escalation notes""",
    agent=escalation_agent,
    expected_output="Resolution status + justification."
)

In [ ]:

# ---------------- Crew Setup ----------------
cs_crew = Crew(
    agents=[issue_classifier, knowledge_agent, solution_agent, escalation_agent],
    tasks=[identify_issue, fetch_knowledge, generate_solution_response, apply_escalation_logic],
    verbose=True
)

2025-12-09 20:49:02,963 - 21008 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [56]:
# ---------------- Run ----------------
# customer_query = input("\nEnter customer message: ")
customer_query= """I’m unable to log into my account even though I’m sure the password is correct. Can someone help?"""

result = cs_crew.kickoff(inputs={"customer_query": customer_query})

print("\n--- FINAL SYSTEM RESPONSE ---\n")
print(result)

 [DEBUG]: == Working Agent: Issue Identification Specialist
 [INFO]: == Starting Task: Analyze the customer message: "I’m unable to log into my account even though I’m sure the password is correct. Can someone help?".
Identify:
- Issue type (billing, technical, login, refund, general inquiry)
- Sentiment (angry, neutral, confused, urgent)
- Priority level (low, normal, high)
Output as JSON with keys: type, urgency, sentiment, summary.


> Entering new CrewAgentExecutor chain...
The customer is experiencing an issue with logging into their account, which indicates a technical problem related to account access. The customer expresses certainty about their password being correct, suggesting they may be confused about why they cannot log in. The tone of the message does not indicate anger but rather a request for assistance, which leans towards a neutral sentiment. Given that the customer is unable to access their account, this issue should be treated with a normal priority level.

Action:

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
# This is urgent. My business is down because your system won’t sync orders. Fix this ASAP.
# I was charged twice for my subscription this month. Please refund the extra amount.
# The app keeps crashing every time I try to open the dashboard. This started after the latest update.
# Does your software support exporting data to Excel? I can’t find the option anywhere.
